# Python DAQ (Arduino Uno → Jupyter)
Live plot + CSV logging using **pyserial + matplotlib**.

**Workflow**
1. Upload Arduino sketch that streams `us,raw` lines over Serial.
2. Run notebook cells top-to-bottom.
3. When finished, run the **Stop + Export** cell to close the port and write the CSV.

> Tip: Close the Arduino Serial Monitor while running Python (the port is often exclusive).


## 0) Install / environment notes
If you’re using **Anaconda**, you can install the interactive matplotlib backend (recommended) once:

```bash
conda install -c conda-forge ipympl pyserial pandas matplotlib
```

Then restart the kernel.

If `%matplotlib widget` fails, try `%matplotlib notebook` (classic Notebook) or use the script-based approach.


In [ ]:
# Interactive plotting backend (best for live updates in Jupyter)
%matplotlib widget

import time
import threading
from collections import deque

import serial
from serial.tools import list_ports
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import numpy as np

Helper functions (Hidden, but must run)

In [ ]:
#-- Function: parse_line ---
def parse_line(line: str):
    """
    Arduino output can include:
      READY
      arduino_us,raw   (header)
      <arduino_us>,<raw>
      DONE

    Returns:
      ("DATA", arduino_us:int, raw:int) or
      ("READY"/"HEADER"/"DONE", None, None) or
      None (ignore)
    """
    s = line.strip()
    if not s:
        return None

    sl = s.lower()

    if sl == "ready":
        return ("READY", None, None)

    if sl == "done":
        return ("DONE", None, None)

    if sl.startswith("arduino_us") or sl.startswith("us"):
        return ("HEADER", None, None)

    parts = s.split(",")
    if len(parts) != 2:
        return None

    try:
        arduino_us = int(parts[0].strip())
        raw = int(parts[1].strip())
        return ("DATA", arduino_us, raw)
    except ValueError:
        return None

#-- Function: reader()--
def reader():
    """
    Background thread: reads serial lines and logs exactly n_samples DATA samples.
    Uses parse_line() that returns (kind, arduino_us, raw) or None.
    """
    global stop_flag

    t0_arduino_us = None

    while not stop_flag and len(log_rows) < n_samples:
        line = ser.readline().decode("utf-8", errors="ignore")
        parsed = parse_line(line)
        if parsed is None:
            continue

        kind, arduino_us, raw = parsed

        # Ignore non-data messages (but optionally print once for debugging)
        if kind == "READY":
            continue
        if kind == "HEADER":
            continue
        if kind == "DONE":
            # Arduino says it's finished early; stop cleanly
            stop_flag = True
            break

        # Data case
        # (kind == "DATA")
        if t0_arduino_us is None:
            t0_arduino_us = arduino_us

        t_s = (arduino_us - t0_arduino_us) / 1_000_000.0
        volts = raw * (v_ref / adc_max)

        tbuf.append(t_s)
        vbuf.append(volts)

        log_rows.append({
            "pc_time_s": time.time() - t0_pc,
            "arduino_us": arduino_us,
            "raw": raw,
            "volts": volts
        })

    stop_flag = True
    
    # Extract Arduino timestamps (microseconds)
    arduino_us = np.array([r["arduino_us"] for r in log_rows], dtype=np.int64)

    # Compute sample-to-sample intervals
    dt_us = np.diff(arduino_us)          # microseconds
    dt_s = dt_us * 1e-6                  # seconds

    # Statistics
    median_dt_s = float(np.median(dt_s))
    stdev_dt_s  = float(np.std(dt_s, ddof=1)) if len(dt_s) > 1 else 0.0
    median_freq_hz = 1.0 / median_dt_s if median_dt_s > 0 else float("nan")

    print("=== Sampling statistics ===")
    print(f"Samples collected: {len(arduino_us)}")
    print(f"        Median Δt: {median_dt_s:.6f} s")
    print(f"         Δt stdev: {stdev_dt_s:.6f} s")
    print(f" Median frequency: {median_freq_hz:.1f} Hz")

## 1) List available serial ports
Unplug/replug the Arduino if you’re unsure which port is yours (the new port that appears is usually the Arduino).


In [ ]:
ports = list(list_ports.comports())
if not ports:
    print("No serial ports found.")
else: 
    print("Detected serial ports:")
    for i, p in enumerate(ports):
        desc = (p.description or "").strip()
        manuf = (p.manufacturer or "").strip()
        print(f"  [{i}] {p.device:>12}  | {desc}  | {manuf} ")

## 2) Connect to the Arduino
Edit `PORT` to match your Arduino port from the list above.

- macOS often looks like: `/dev/cu.usbmodemXXXX`
- Windows often looks like: `COM5`

Also check that `BAUD` matches the baud rate specified in `Serial.begin()` in your Arduino sketch


In [ ]:
# --- EDIT THIS ---
PORT = "/dev/cu.usbmodem101"   # e.g., "COM5" on Windows
BAUD = 115200                  # must match your Arduino Sketch

#--- Connection Test---
# Connect (opening the port typically resets the Arduino)
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2.0)   # allow Arduino reset to finish
ser.reset_input_buffer()

print(f"Connected to {PORT} at {BAUD} baud")
print("Testing communication...")

# --- Send START command ---
sample_rate_hz =100
n_samples = 5
cmd = f"START,{sample_rate_hz},{n_samples}\n"
ser.write(cmd.encode("utf-8"))
ser.flush()
print("Sent:", cmd.strip())

for _ in range(7):
    line = ser.readline().decode("utf-8", errors="ignore")
    parsed = parse_line(line)
    if parsed is None:
        continue

    kind, arduino_us, raw = parsed
    print(f"{kind}: {arduino_us}, {raw}")

## 4) Start acquisition (background thread)
This cell starts a background reader thread that:
- reads serial lines (`us,raw`)
- fills plotting buffers (last *N* points)
- logs the full run into `log_rows` for CSV export

If your Arduino prints a header like `us,raw`, it will be ignored automatically.


In [ ]:
#--- HARDWARE SETUP, EDIT THIS ---

duration_s = 15.0               # How Long to gather data [seconds]
sample_rate_hz = 50            # Desired Sample Rate in hZ

v_ref = 5.0                     # ADC reference voltage (V) 5V on Uno. 3.3V on Teensy4.1
adc_max = 1023                  # 10-bit ADC on Uno (2^10-1). 12 Bit on Teensy (2^12-1)

plot_window_s = 1             # seconds shown in the live view window

# --- Clear buffers / state ---
n_samples = int(round(sample_rate_hz * duration_s))
N = int(sample_rate_hz*plot_window_s) #buffer size for live view
tbuf = deque(maxlen=N)
vbuf = deque(maxlen=N)
log_rows = []
stop_flag = False
t0_pc = time.time()

# --- (Re)connect + reset (optional but good for repeatability) ---
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2.0)   # allow Arduino reset to finish
ser.reset_input_buffer()
print(f"Connected to {PORT} at {BAUD} baud")

# --- Send START command to Arduino ---
cmd = f"START,{sample_rate_hz},{n_samples}\n"
ser.write(cmd.encode("utf-8"))
ser.flush()
print("Sent:", cmd.strip())

thread = threading.Thread(target=reader, daemon=False)  ## Uses Reader Thread defined up top!
thread.start()
print("Reader thread started.")

# --- The Code below is for the Live plot of data

fig, ax = plt.subplots()
(line_plot,) = ax.plot([], [])
ax.set_xlabel("Arduino time (s)")
ax.set_ylabel("Volts (V)")
ax.grid(True)
ax.set_title("Live DAQ (Arduino → Python)")

def init():
    line_plot.set_data([], [])
    return (line_plot,)

def update(_frame):
    if len(tbuf) < 2:
        return (line_plot,)

    x = list(tbuf)
    y = list(vbuf)

    line_plot.set_data(x, y)
    ax.relim()
    ax.autoscale_view()
    return (line_plot,)

ani = FuncAnimation(
    fig,
    update,
    init_func=init,
    interval=50,
    blit=False,
    cache_frame_data=False
)

plt.show()

## 6) Stop + export to CSV ✅
Run this when you’re done collecting data.

This cell:
- stops acquisition thread
- stops animation
- closes the serial port
- closes the figure (important for Jupyter)
- writes the full log to CSV


In [ ]:
# 1) Stop acquisition
stop_flag = True
thread.join(timeout=1)

# 2) Stop animation (prevents the kernel from staying "busy")
try:
    ani.event_source.stop()
except Exception:
    pass

# 3) Close serial
try:
    ser.close()
except Exception:
    pass

# 4) Close figure
plt.close(fig)

# 5) Export CSV
df = pd.DataFrame(log_rows)

timestamp = time.strftime("%Y%m%d_%H%M%S")
csv_name = f"daq_data_{timestamp}.csv"

df.to_csv(csv_name, index=False)
print(f"Saved {len(df)} samples to {csv_name}")

df.head()

## 7) (Optional) Quick post-run plot
This is a static plot of the full run (after Stop + Export).


In [ ]:
if "df" in globals() and len(df) > 0:
    plt.figure()
    plt.plot(df["arduino_us"] / 1_000_000.0, df["volts"])
    plt.xlabel("Arduino time (s)")
    plt.ylabel("Volts (V)")
    plt.title("DAQ run (static plot)")
    plt.grid(True)
    plt.show()
else:
    print("No data captured (or df not created yet). Run Stop + Export first.")